In [ ]:
import cv2
from ultralytics import YOLO
import scrcpy.core as scrcpy
import tkinter as tk
import customtkinter as ctk
import utils
from PIL import Image

In [ ]:
img_scale = 0.3
update_timer = 1
client = utils.connect_to_device("127.0.0.1:5037")
# Load the YOLOv8 model
model = YOLO("yolo_models/yolov8n.pt")
model.to("cuda:0")
feed_res = (int(client.resolution[0] * img_scale), int(client.resolution[1] * img_scale))

In [ ]:
class PokemonBattleAssistant(ctk.CTk):
    def __init__(self, feed_res):
        super().__init__()
        self.title("Pokemon Battle Assistant")
        self.feed_res = feed_res

        mainframe = ctk.CTkFrame(self)
        mainframe.grid(column=0, row=0, sticky=(tk.W, tk.E, tk.N, tk.S), padx=30, pady=20)

        pil_image = Image.new("RGB", feed_res)
        self.my_image = ctk.CTkImage(light_image=pil_image, dark_image=pil_image, size=feed_res)
        self.image_label = ctk.CTkLabel(mainframe, text="", image=self.my_image)
        self.image_label.grid(column=0, row=3, pady=0)
        self.screen = None

    def yolo_inference(self, frame):
        # Perform inference
        self.screen = frame.copy()  # Create a copy of the frame
        results = model(self.screen, verbose=False)
        # Get the bounding box data
        bbox_data = results[0].boxes.data

        # Map class IDs to class names
        class_names = results[0].names

        # Check if there are any detections
        if bbox_data.shape[0] > 0:
            for bbox in bbox_data:
                # Extract bounding box coordinates
                x1, y1, x2, y2, conf, cls = bbox.tolist()

                # Convert coordinates to integers
                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

                # Draw bounding box on the image
                cv2.rectangle(self.screen, (x1, y1), (x2, y2), (0, 255, 0), 5)

                # Draw class name and confidence on the image
                class_name = class_names[int(cls)]
                cv2.putText(
                    self.screen,
                    f"{class_name} {conf:.2f}",
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.9,
                    (0, 255, 0),
                    5,
                )

    def update_ui(self, client):
        original_screen = client.last_frame
        if original_screen is not None:
            self.yolo_inference(original_screen)
            resized_image = cv2.resize(self.screen, self.feed_res, interpolation=cv2.INTER_AREA)
            resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(resized_image)
            self.my_image.configure(light_image=pil_img, dark_image=pil_img)

        app.after(update_timer, lambda: self.update_ui(client))


if __name__ == "__main__":
    app = PokemonBattleAssistant(feed_res)
    app.update_ui(client)
    app.mainloop()